## Federated Learning with one corrupted node: krum
Using one balanced and one unbalanced dataset with one corrupted node (5%, 25% and 50% corrupted samples) to test different aggregation functions and determine the more robust one. 

In [1]:
# Imports
import tensorflow as tf
import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, BatchNormalization
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import KFold

# Disable warns
pd.options.mode.chained_assignment = None  # default='warn'

In [2]:
def preprocessing(data): 

    # Select the 'proto' and 'state' values that I want
    data = data.loc[(data['proto'] == 'tcp') | (data['proto'] =='udp') | (data['proto'] =='icmp') | (data['proto'] =='arp') | (data['proto'] =='ipv6-icmp') | (data['proto'] =='igmp') | (data['proto'] =='rarp'), :]
    data = data.loc[(data['state'] == 'RST') | (data['state'] =='REQ') | (data['state'] =='INT') | (data['state'] =='FIN') | (data['state'] =='CON') | (data['state'] =='ECO') | (data['state'] =='ACC') | (data['state'] == 'PAR'), :]

    # Extracting labels 
    data_labels = data[['label']]

    # Drop the invalid features and select interested data features
    data_features=data[['proto','srcip','sport','dstip','dsport','spkts','dpkts','sbytes','dbytes','state','stime','ltime','dur']]

    """PREPROCESSING"""


    # Preprocess IP and ports features
    # IP Source Address
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(".")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: x.split(":")[-1])
    data_features['srcip'] = data_features['srcip'].apply(lambda x: int(x, 16))


    # IP Destination Address
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(".")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: x.split(":")[-1])
    data_features['dstip'] = data_features['dstip'].apply(lambda x: int(x, 16))

    # Ports
    data_features['sport'] = data_features['sport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)
    data_features['dsport'] = data_features['dsport'].apply(lambda x: x.replace('0x','') if "0x" in str(x) else x)

    # Convert all ports with 0 decimal, and HEX to DEC
    data_features['sport'] = data_features['sport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['sport'] = data_features['sport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    data_features['dsport'] = data_features['dsport'].apply(lambda x: str(x)[:-2] if str(x)[-2:] == '.0' else str(x))
    data_features['dsport'] = data_features['dsport'].apply(lambda x: -1 if str(x).isalpha()==True else int(x,16))

    # Convert field to int format
    data_features['srcip'] = data_features['srcip'].astype(int)
    data_features['sport'] = data_features['sport'].astype(int)
    data_features['dstip'] = data_features['dstip'].astype(int)
    data_features['dsport'] = data_features['dsport'].astype(int)

    # Convert some fields to logarithmic
    log1p_col = ['dur', 'sbytes', 'dbytes', 'spkts']

    for col in log1p_col:
        data_features[col] = data_features[col].apply(np.log1p)

    # Create a complementary field of attack & Transform to One hot encoding - LABELS
    normal=data_labels['label']
    normal=normal.replace(1,2)
    normal=normal.replace(0,1)
    normal=normal.replace(2,0)

    # Insert the new column in data labels
    data_labels.insert(1, 'normal', normal)
    data_labels = pd.get_dummies(data_labels)

    data_labels = pd.get_dummies(data_labels)

    # Transform to One hot encoding - FEATURES
    data_features=pd.get_dummies(data_features)

    # Value given for the missing columns
    auxCol=0

    # As we are using different datasets that might not have all representations, we are going to detect and add the missing columns 
    # The columns that can have types are: proto and state: need to check if all representations are done 
    state_cols = [col for col in data_features if col.startswith('state_')]
    proto_cols = [col for col in data_features if col.startswith('proto_')]
    
    # Check if all columns are present
    if 'state_PAR' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_PAR', auxCol, True)
    if 'state_ACC' not in state_cols: 
        data_features.insert(data_features.shape[1], 'state_ACC', auxCol, True)
    if 'state_ECO' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_ECO', auxCol, True)
    if 'state_CON' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_CON', auxCol, True)
    if 'state_FIN' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_FIN', auxCol, True)
    if 'state_INT' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_INT', auxCol, True)
    if 'state_REQ' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_REQ', auxCol, True)
    if 'state_RST' not in state_cols:
        data_features.insert(data_features.shape[1], 'state_RST', auxCol, True)
    if 'proto_igmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_igmp', auxCol, True)
    if 'proto_arp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_arp', auxCol, True)
    if 'proto_icmp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_icmp', auxCol, True)
    if 'proto_udp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_udp', auxCol, True)
    if 'proto_tcp' not in proto_cols:
        data_features.insert(data_features.shape[1], 'proto_tcp', auxCol, True)

    # Normalize all data features
    data_features = StandardScaler().fit_transform(data_features)

    #Add dimension to data features
    data_features = np.expand_dims(data_features, axis=2)
    data_features = np.expand_dims(data_features, axis=3)

    x = data_features
    y = data_labels.to_numpy()

    return x, y

In [3]:
# Model building and definition
def build_model(input_shape):
    model = models.Sequential()
    model.add(layers.Conv2D(filters=32,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Conv2D(filters=64,  input_shape=input_shape, kernel_size=(1,10), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D(pool_size=(1, 1), padding='same'))
    model.add(layers.Flatten())
    model.add(Dense(444, activation='relu'))
    model.add(Dense(2, activation='softmax'))

    return model 

In [4]:
# Returns values of loss, accuracy, f1, precision and recall of model evaluating with test dataset 
def evaluation(model, x, y): 
    loss, accuracy = model.evaluate(x, y)
    y_pred = model.predict(x)
    y_pred = np.argmax(y_pred, axis=1)
    y = np.argmax(y, axis=1)
    report = classification_report(y, y_pred, target_names=['normal', 'attack'], output_dict=True)
    # Obtain f1, precision and recall from the report
    f1 = report['weighted avg']['f1-score']
    precision = report['weighted avg']['precision']
    recall = report['weighted avg']['recall']
    return loss, accuracy, f1, precision, recall

In [5]:
def aggregate(grad_list, num_mal = 0):
    
    num_to_consider = num_nodes - num_mal - 2

    # Flatten gradients to compute distances
    flat_grads = [tf.concat([tf.reshape(g, [-1]) for g in grad], axis=0).numpy() for grad in grad_list]

    # Compute pairwise squared Euclidean distances
    distances = np.zeros((num_nodes, num_nodes))
    for i in range(num_nodes):
        for j in range(i + 1, num_nodes):
            dist = np.sum((flat_grads[i] - flat_grads[j]) ** 2)
            distances[i, j] = dist
            distances[j, i] = dist

    # Find the Krum gradient
    krum_scores = []
    for i in range(num_nodes):
        sorted_distances = np.sort(distances[i])
        score = np.sum(sorted_distances[:num_to_consider])
        krum_scores.append(score)
    
    krum_index = np.argmin(krum_scores)
    selected_grad = grad_list[krum_index]

    return selected_grad

In [6]:
test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')
test_plus = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test+.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3836997398.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  test_basic = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Test-Basic.csv')


In [7]:
xbasic, ybasic = preprocessing(test_basic)
xplus, yplus = preprocessing(test_plus)

#### 5A 5% 

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3687049449.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3687049449.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3687049449.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3687049449.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on im

In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Akrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Akrum.hdf5')

Epoch 1/5
31/31 [==============================] - 9s 275ms/step - loss: 0.4649 - accuracy: 0.8491 - val_loss: 0.2441 - val_accuracy: 0.9182
Epoch 2/5
31/31 [==============================] - 7s 232ms/step - loss: 0.1929 - accuracy: 0.9522 - val_loss: 0.1806 - val_accuracy: 0.9603
Epoch 3/5
31/31 [==============================] - 8s 270ms/step - loss: 0.1678 - accuracy: 0.9647 - val_loss: 0.1640 - val_accuracy: 0.9631
Epoch 4/5
31/31 [==============================] - 9s 277ms/step - loss: 0.1556 - accuracy: 0.9664 - val_loss: 0.1548 - val_accuracy: 0.9646
Epoch 5/5
31/31 [==============================] - 9s 293ms/step - loss: 0.1478 - accuracy: 0.9674 - val_loss: 0.1493 - val_accuracy: 0.9657
Epoch 1/5
31/31 [==============================] - 12s 384ms/step - loss: 0.0338 - accuracy: 0.9896 - val_loss: 0.4060 - val_accuracy: 0.7461
Epoch 2/5
31/31 [==============================] - 9s 302ms/step - loss: 0.0236 - accuracy: 0.9906 - val_loss: 0.3226 - val_accuracy: 0.7786
Epoch 3/5
31

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04024306684732437, 0.3410366177558899], [0.059216126799583435, 0.5945024490356445], [0.07099304348230362, 0.5482661724090576]]
Accuracy for iterations:  [[0.9809007048606873, 0.8243115544319153], [0.9721946120262146, 0.7246057987213135], [0.9706535339355469, 0.7562486529350281]]
F1 for iterations:  [[0.9808898894118244, 0.8155998058822741], [0.9721490752024962, 0.691143612706857], [0.9705968326335274, 0.7332728705765952]]
Precision for iterations:  [[0.9810223768682153, 0.8577772105090989], [0.9729510504596791, 0.8047463105863021], [0.971636109762449, 0.8198943188648957]]
Recall for iterations:  [[0.9809006982383359, 0.8243115599796556], [0.9721946302842619, 0.7246058272179031], [0.9706535393964183, 0.7562486376516748]]


#### 5B 10%

In [16]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/2463029928.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/2463029928.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/2463029928.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/2463029928.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [17]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [18]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Akrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [19]:
global_model = build_model((24,1,1))

In [20]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [21]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [22]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Akrum.hdf5')

Epoch 1/5
31/31 [==============================] - 8s 258ms/step - loss: 0.4891 - accuracy: 0.8099 - val_loss: 0.3006 - val_accuracy: 0.8955
Epoch 2/5
31/31 [==============================] - 10s 307ms/step - loss: 0.2618 - accuracy: 0.9315 - val_loss: 0.2424 - val_accuracy: 0.9406
Epoch 3/5
31/31 [==============================] - 10s 322ms/step - loss: 0.2386 - accuracy: 0.9413 - val_loss: 0.2308 - val_accuracy: 0.9420
Epoch 4/5
31/31 [==============================] - 10s 309ms/step - loss: 0.2296 - accuracy: 0.9425 - val_loss: 0.2259 - val_accuracy: 0.9427
Epoch 5/5
31/31 [==============================] - 10s 329ms/step - loss: 0.2246 - accuracy: 0.9426 - val_loss: 0.2238 - val_accuracy: 0.9428
Epoch 1/5
31/31 [==============================] - 13s 413ms/step - loss: 0.0396 - accuracy: 0.9896 - val_loss: 0.6025 - val_accuracy: 0.6315
Epoch 2/5
31/31 [==============================] - 13s 414ms/step - loss: 0.0237 - accuracy: 0.9906 - val_loss: 0.2929 - val_accuracy: 0.8264
Epoch 3

In [23]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.04719569534063339, 0.3650592863559723], [0.0888303890824318, 0.5638192296028137], [0.06911387294530869, 0.3612609803676605]]
Accuracy for iterations:  [[0.9742177724838257, 0.8103247880935669], [0.9644672870635986, 0.7493097186088562], [0.9715153574943542, 0.8018419146537781]]
F1 for iterations:  [[0.9741852704659083, 0.7994188036670806], [0.9643681437205753, 0.7241521488267526], [0.9714632203775988, 0.7895104623190683]]
Precision for iterations:  [[0.9747099407064148, 0.8494828961826225], [0.9661703450371655, 0.817042393301229], [0.9724199668271004, 0.8441920596159352]]
Recall for iterations:  [[0.9742177685588244, 0.8103247838407324], [0.9644672646001928, 0.749309743515222], [0.9715153816938853, 0.8018418949356971]]


#### 5B 25%

In [24]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3855986297.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3855986297.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3855986297.py:4: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15A-Part4.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_8464/3855986297.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on

In [25]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [26]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Akrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [27]:
global_model = build_model((24,1,1))

In [28]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [29]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [30]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Akrum.hdf5')

Epoch 1/5
31/31 [==============================] - 10s 314ms/step - loss: 0.4356 - accuracy: 0.8797 - val_loss: 0.1905 - val_accuracy: 0.9499
Epoch 2/5
31/31 [==============================] - 9s 306ms/step - loss: 0.1486 - accuracy: 0.9646 - val_loss: 0.1265 - val_accuracy: 0.9752
Epoch 3/5
31/31 [==============================] - 9s 299ms/step - loss: 0.1202 - accuracy: 0.9766 - val_loss: 0.1114 - val_accuracy: 0.9774
Epoch 4/5
31/31 [==============================] - 9s 296ms/step - loss: 0.1058 - accuracy: 0.9788 - val_loss: 0.1034 - val_accuracy: 0.9788
Epoch 5/5
31/31 [==============================] - 9s 298ms/step - loss: 0.0993 - accuracy: 0.9798 - val_loss: 0.0990 - val_accuracy: 0.9790
Epoch 1/5
31/31 [==============================] - 14s 466ms/step - loss: 0.0341 - accuracy: 0.9895 - val_loss: 0.3658 - val_accuracy: 0.7673
Epoch 2/5
31/31 [==============================] - 13s 415ms/step - loss: 0.0230 - accuracy: 0.9910 - val_loss: 0.4397 - val_accuracy: 0.6656
Epoch 3/5


In [31]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.058896299451589584, 0.4204658567905426], [0.060395222157239914, 0.3623192608356476], [0.07335920631885529, 0.31031617522239685]]
Accuracy for iterations:  [[0.9715518951416016, 0.7891448140144348], [0.972355306148529, 0.8107243776321411], [0.9727277755737305, 0.8455097079277039]]
F1 for iterations:  [[0.9715039140687096, 0.7742793456575021], [0.9723113870920939, 0.7997299547989344], [0.9726827033508688, 0.8393311537717432]]
Precision for iterations:  [[0.9723428089355969, 0.8372615773079198], [0.9730749416431967, 0.850605211982617], [0.973498221066896, 0.8722525917485232]]
Recall for iterations:  [[0.9715519004353034, 0.7891448085446487], [0.9723553127465016, 0.8107244060161302], [0.9727278039089661, 0.8455096999200755]]


#### 5B 5% 

In [8]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3503541895.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3503541895.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3503541895.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15B-Part5.csv')


In [9]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [10]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55Bkrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [11]:
global_model = build_model((24,1,1))

In [12]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [13]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [14]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list, 1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55Bkrum.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 184ms/step - loss: 0.4548 - accuracy: 0.8325 - val_loss: 0.2270 - val_accuracy: 0.9457
Epoch 2/5
31/31 [==============================] - 5s 163ms/step - loss: 0.1945 - accuracy: 0.9523 - val_loss: 0.1711 - val_accuracy: 0.9627
Epoch 3/5
31/31 [==============================] - 5s 165ms/step - loss: 0.1662 - accuracy: 0.9647 - val_loss: 0.1590 - val_accuracy: 0.9649
Epoch 4/5
31/31 [==============================] - 5s 163ms/step - loss: 0.1537 - accuracy: 0.9666 - val_loss: 0.1504 - val_accuracy: 0.9658
Epoch 5/5
31/31 [==============================] - 5s 161ms/step - loss: 0.1462 - accuracy: 0.9675 - val_loss: 0.1460 - val_accuracy: 0.9660
Epoch 1/5
31/31 [==============================] - 5s 169ms/step - loss: 0.0337 - accuracy: 0.9902 - val_loss: 0.4542 - val_accuracy: 0.7043
Epoch 2/5
31/31 [==============================] - 5s 159ms/step - loss: 0.0231 - accuracy: 0.9914 - val_loss: 0.3972 - val_accuracy: 0.6865
Epoch 3/5
31/

In [15]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.1684008687734604, 0.40952253341674805], [0.1413555145263672, 0.3790534436702728], [0.1160159781575203, 0.39062780141830444]]
Accuracy for iterations:  [[0.9237927198410034, 0.7483288645744324], [0.933871865272522, 0.8059471249580383], [0.945258378982544, 0.7962108254432678]]
F1 for iterations:  [[0.9230588667745264, 0.7225935812034462], [0.9333646098000338, 0.794245833679687], [0.9449435130368761, 0.7826309359104554]]
Precision for iterations:  [[0.9324646335626502, 0.8178620846368236], [0.9404585965960172, 0.847160974044791], [0.9498496010451459, 0.8419750922243834]]
Recall for iterations:  [[0.9237926904087178, 0.7483288527210638], [0.9338718630401122, 0.8059471045556929], [0.9452584066142744, 0.7962108551914554]]


#### 5B 10%

In [23]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/2097221778.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/2097221778.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/2097221778.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/2097221778.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [24]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [25]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105Bkrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [26]:
global_model = build_model((24,1,1))

In [27]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [28]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [29]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105Bkrum.hdf5')

Epoch 1/5
31/31 [==============================] - 6s 180ms/step - loss: 0.5200 - accuracy: 0.7977 - val_loss: 0.3249 - val_accuracy: 0.8611
Epoch 2/5
31/31 [==============================] - 6s 178ms/step - loss: 0.2770 - accuracy: 0.9201 - val_loss: 0.2488 - val_accuracy: 0.9387
Epoch 3/5
31/31 [==============================] - 5s 172ms/step - loss: 0.2454 - accuracy: 0.9391 - val_loss: 0.2326 - val_accuracy: 0.9418
Epoch 4/5
31/31 [==============================] - 5s 176ms/step - loss: 0.2334 - accuracy: 0.9413 - val_loss: 0.2253 - val_accuracy: 0.9431
Epoch 5/5
31/31 [==============================] - 5s 176ms/step - loss: 0.2260 - accuracy: 0.9424 - val_loss: 0.2205 - val_accuracy: 0.9433
Epoch 1/5
31/31 [==============================] - 5s 174ms/step - loss: 0.0399 - accuracy: 0.9892 - val_loss: 0.3696 - val_accuracy: 0.7791
Epoch 2/5
31/31 [==============================] - 5s 171ms/step - loss: 0.0238 - accuracy: 0.9911 - val_loss: 0.3963 - val_accuracy: 0.7012
Epoch 3/5
31/

In [30]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13186179101467133, 0.28262853622436523], [0.11804281920194626, 0.30547288060188293], [0.09732036292552948, 0.274226576089859]]
Accuracy for iterations:  [[0.9240337014198303, 0.808708131313324], [0.9356174468994141, 0.8282532691955566], [0.9467775821685791, 0.8518854975700378]]
F1 for iterations:  [[0.9233071891439947, 0.7974891501913676], [0.9351506975574713, 0.8200429427291401], [0.946498396984034, 0.8464938030895175]]
Precision for iterations:  [[0.9326330798875049, 0.8487547487978524], [0.9417476897623043, 0.8605265809104845], [0.9508262541299596, 0.8757591249282343]]
Recall for iterations:  [[0.9240337141020771, 0.8087081304948049], [0.9356174588798971, 0.8282532878006249], [0.9467775862572673, 0.8518854900821042]]


#### 5B 25%

In [31]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3291834707.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part1.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3291834707.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part2.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3291834707.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15B-Part3.csv')
C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/3291834707.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype optio

In [32]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [33]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255Bkrum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [34]:
global_model = build_model((24,1,1))

In [35]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [36]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [37]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255Bkrum.hdf5')

Epoch 1/5
31/31 [==============================] - 5s 167ms/step - loss: 0.5465 - accuracy: 0.7763 - val_loss: 0.4250 - val_accuracy: 0.8530
Epoch 2/5
31/31 [==============================] - 5s 171ms/step - loss: 0.4218 - accuracy: 0.8582 - val_loss: 0.4041 - val_accuracy: 0.8687
Epoch 3/5
31/31 [==============================] - 5s 163ms/step - loss: 0.4073 - accuracy: 0.8661 - val_loss: 0.3949 - val_accuracy: 0.8695
Epoch 4/5
31/31 [==============================] - 5s 164ms/step - loss: 0.3992 - accuracy: 0.8679 - val_loss: 0.3901 - val_accuracy: 0.8705
Epoch 5/5
31/31 [==============================] - 5s 171ms/step - loss: 0.3948 - accuracy: 0.8683 - val_loss: 0.3885 - val_accuracy: 0.8705
Epoch 1/5
31/31 [==============================] - 5s 163ms/step - loss: 0.0557 - accuracy: 0.9899 - val_loss: 0.5896 - val_accuracy: 0.6540
Epoch 2/5
31/31 [==============================] - 5s 171ms/step - loss: 0.0248 - accuracy: 0.9908 - val_loss: 0.3272 - val_accuracy: 0.7973
Epoch 3/5
31/

In [38]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.13437610864639282, 0.2682327926158905], [0.12832242250442505, 0.28907695412635803], [0.12117831408977509, 0.29370149970054626]]
Accuracy for iterations:  [[0.9238657355308533, 0.8304330706596375], [0.9255675077438354, 0.8179539442062378], [0.9290294647216797, 0.8316682577133179]]
F1 for iterations:  [[0.9231360898246124, 0.8226153709281016], [0.9248794591083128, 0.8079918814333199], [0.9284234919289402, 0.82375894040829]]
Precision for iterations:  [[0.93248409535391, 0.8612842189890098], [0.9338233596985744, 0.8557059456721707], [0.9365301622224658, 0.863652215937997]]
Recall for iterations:  [[0.923865727891554, 0.8304330451209765], [0.9255675012416372, 0.8179539344619632], [0.9290294779280727, 0.8316682409358425]]


#### 5C NODE 1 5%

In [39]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/591709827.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%15C-Part5.csv')


In [40]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [41]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C1krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [42]:
global_model = build_model((24,1,1))

In [43]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [44]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [45]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C1krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 173ms/step - loss: 0.6064 - accuracy: 0.7576 - val_loss: 0.4718 - val_accuracy: 0.8035
Epoch 2/5
16/16 [==============================] - 3s 176ms/step - loss: 0.3617 - accuracy: 0.8703 - val_loss: 0.2506 - val_accuracy: 0.9255
Epoch 3/5
16/16 [==============================] - 3s 163ms/step - loss: 0.2257 - accuracy: 0.9387 - val_loss: 0.2012 - val_accuracy: 0.9517
Epoch 4/5
16/16 [==============================] - 3s 170ms/step - loss: 0.1975 - accuracy: 0.9567 - val_loss: 0.1817 - val_accuracy: 0.9622
Epoch 5/5
16/16 [==============================] - 3s 169ms/step - loss: 0.1823 - accuracy: 0.9617 - val_loss: 0.1695 - val_accuracy: 0.9643
Epoch 1/5
14/14 [==============================] - 2s 165ms/step - loss: 0.0418 - accuracy: 0.9897 - val_loss: 0.6628 - val_accuracy: 0.7272
Epoch 2/5
14/14 [==============================] - 2s 171ms/step - loss: 0.0224 - accuracy: 0.9949 - val_loss: 0.8043 - val_accuracy: 0.7270
Epoch 3/5
14/

In [46]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08662975579500198, 0.19675715267658234], [0.04885745048522949, 0.10433225333690643], [0.043407466262578964, 0.10013923794031143]]
Accuracy for iterations:  [[0.9837710857391357, 0.9249255061149597], [0.9903371334075928, 0.9821622967720032], [0.9903590679168701, 0.9806909561157227]]
F1 for iterations:  [[0.9837713844765796, 0.9241394587228892], [0.9903415659214462, 0.9821832394220688], [0.9903634698516782, 0.9806997341844538]]
Precision for iterations:  [[0.9837719204280907, 0.9296400775761161], [0.9905250878553817, 0.9824197982204333], [0.9905472636848005, 0.9807371103277795]]
Recall for iterations:  [[0.9837710713137983, 0.9249255249582213], [0.9903371410207719, 0.9821623192617889], [0.9903590522656227, 0.9806909830705515]]


#### 5C NODE 1 10%

In [47]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/372464333.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%15C-Part5.csv')


In [48]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [49]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C1krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [50]:
global_model = build_model((24,1,1))

In [51]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [52]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [53]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C1krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 170ms/step - loss: 0.6147 - accuracy: 0.7564 - val_loss: 0.4928 - val_accuracy: 0.7759
Epoch 2/5
16/16 [==============================] - 3s 176ms/step - loss: 0.3927 - accuracy: 0.8574 - val_loss: 0.3094 - val_accuracy: 0.9054
Epoch 3/5
16/16 [==============================] - 3s 160ms/step - loss: 0.2932 - accuracy: 0.9179 - val_loss: 0.2806 - val_accuracy: 0.9312
Epoch 4/5
16/16 [==============================] - 3s 177ms/step - loss: 0.2683 - accuracy: 0.9337 - val_loss: 0.2646 - val_accuracy: 0.9381
Epoch 5/5
16/16 [==============================] - 3s 160ms/step - loss: 0.2549 - accuracy: 0.9371 - val_loss: 0.2501 - val_accuracy: 0.9395
Epoch 1/5
14/14 [==============================] - 2s 171ms/step - loss: 0.0468 - accuracy: 0.9909 - val_loss: 0.7060 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 164ms/step - loss: 0.0233 - accuracy: 0.9945 - val_loss: 0.8189 - val_accuracy: 0.7270
Epoch 3/5
14/

In [54]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.08986923098564148, 0.18257004022598267], [0.056640248745679855, 0.11619224399328232], [0.054749954491853714, 0.09699279069900513]]
Accuracy for iterations:  [[0.9851734042167664, 0.9295756816864014], [0.9896798133850098, 0.9688476324081421], [0.9904612898826599, 0.9792196750640869]]
F1 for iterations:  [[0.9851751497575643, 0.9289185993225917], [0.9896847079623124, 0.9688647354430518], [0.9904656539231713, 0.9792450871503925]]
Precision for iterations:  [[0.9851858951589497, 0.9335531314834116], [0.9898860415960865, 0.96892390353904], [0.990647201756734, 0.9795069419566703]]
Recall for iterations:  [[0.9851733909842532, 0.9295756739083049], [0.9896798036752461, 0.9688476349633074], [0.9904613047415934, 0.9792196468793141]]


#### 5C NODE 1 25%

In [55]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/2868219575.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%15C-Part5.csv')


In [56]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [57]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C1krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [58]:
global_model = build_model((24,1,1))

In [59]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [60]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [61]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C1krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 167ms/step - loss: 0.6302 - accuracy: 0.7202 - val_loss: 0.5406 - val_accuracy: 0.7400
Epoch 2/5
16/16 [==============================] - 3s 182ms/step - loss: 0.4878 - accuracy: 0.8044 - val_loss: 0.4417 - val_accuracy: 0.8490
Epoch 3/5
16/16 [==============================] - 3s 162ms/step - loss: 0.4404 - accuracy: 0.8510 - val_loss: 0.4190 - val_accuracy: 0.8646
Epoch 4/5
16/16 [==============================] - 3s 165ms/step - loss: 0.4236 - accuracy: 0.8617 - val_loss: 0.4084 - val_accuracy: 0.8677
Epoch 5/5
16/16 [==============================] - 3s 160ms/step - loss: 0.4139 - accuracy: 0.8639 - val_loss: 0.4017 - val_accuracy: 0.8686
Epoch 1/5
14/14 [==============================] - 2s 160ms/step - loss: 0.0744 - accuracy: 0.9905 - val_loss: 0.5449 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 2s 165ms/step - loss: 0.0258 - accuracy: 0.9938 - val_loss: 0.7649 - val_accuracy: 0.7270
Epoch 3/5
14/

In [62]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.07960396260023117, 0.14799557626247406], [0.07333571463823318, 0.16630017757415771], [0.07445225864648819, 0.14362014830112457]]
Accuracy for iterations:  [[0.9882774949073792, 0.9552241563796997], [0.9869773983955383, 0.9459238648414612], [0.9858234524726868, 0.9553694725036621]]
F1 for iterations:  [[0.9882812829735476, 0.9550575842394174], [0.9869849646502328, 0.9461219806778096], [0.9858321220954305, 0.9555235376094493]]
Precision for iterations:  [[0.9883637640535404, 0.956105398920262], [0.9873245914522379, 0.9518515615657411], [0.9862316971488606, 0.9594860397418689]]
Recall for iterations:  [[0.9882774840047912, 0.9552241517111095], [0.986977416810307, 0.9459238538109423], [0.9858234245814952, 0.9553694688657997]]


#### 5C NODE 3 5%

In [63]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/927562370.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr5%35C-Part5.csv')


In [64]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [65]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr55C3krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [66]:
global_model = build_model((24,1,1))

In [67]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [68]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [69]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr55C3krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 166ms/step - loss: 0.5378 - accuracy: 0.7513 - val_loss: 1.7451 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 175ms/step - loss: 0.2130 - accuracy: 0.9427 - val_loss: 3.6369 - val_accuracy: 0.1252
Epoch 3/5
16/16 [==============================] - 3s 158ms/step - loss: 0.0908 - accuracy: 0.9893 - val_loss: 4.2255 - val_accuracy: 0.1338
Epoch 4/5
16/16 [==============================] - 3s 160ms/step - loss: 0.0559 - accuracy: 0.9893 - val_loss: 2.3574 - val_accuracy: 0.1343
Epoch 5/5
16/16 [==============================] - 3s 165ms/step - loss: 0.0377 - accuracy: 0.9900 - val_loss: 1.3178 - val_accuracy: 0.1912
Epoch 1/5
14/14 [==============================] - 2s 165ms/step - loss: 0.0230 - accuracy: 0.9950 - val_loss: 0.8965 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 181ms/step - loss: 0.0199 - accuracy: 0.9951 - val_loss: 0.5689 - val_accuracy: 0.7291
Epoch 3/5

In [70]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09383776783943176, 0.1116621345281601], [0.1871568113565445, 0.21245206892490387], [0.24568936228752136, 0.23527847230434418]]
Accuracy for iterations:  [[0.9892342686653137, 0.9701191782951355], [0.9627216458320618, 0.9142628908157349], [0.936413586139679, 0.8908305168151855]]
F1 for iterations:  [[0.9892384395018187, 0.9700797572620499], [0.9626600208631052, 0.9131182784715846], [0.9360802316775654, 0.8885853397678093]]
Precision for iterations:  [[0.9893613943206596, 0.9702327402213295], [0.963450010331636, 0.9209371639443125], [0.9402978585222659, 0.902717012577678]]
Recall for iterations:  [[0.9892342750299453, 0.9701191600668458], [0.9627216687604079, 0.9142628787328344], [0.9364135674428117, 0.890830487539054]]


#### 5C NODE 3 10%

In [71]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/4114672810.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr10%35C-Part5.csv')


In [72]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [73]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr105C3krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [74]:
global_model = build_model((24,1,1))

In [75]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [76]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [77]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr105C3krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 181ms/step - loss: 0.5581 - accuracy: 0.7275 - val_loss: 1.5746 - val_accuracy: 0.0000e+00
Epoch 2/5
16/16 [==============================] - 3s 181ms/step - loss: 0.2147 - accuracy: 0.9578 - val_loss: 3.5486 - val_accuracy: 0.1334
Epoch 3/5
16/16 [==============================] - 3s 174ms/step - loss: 0.0899 - accuracy: 0.9892 - val_loss: 3.8852 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 170ms/step - loss: 0.0565 - accuracy: 0.9893 - val_loss: 1.8051 - val_accuracy: 0.1378
Epoch 5/5
16/16 [==============================] - 3s 174ms/step - loss: 0.0405 - accuracy: 0.9886 - val_loss: 1.3167 - val_accuracy: 0.2215
Epoch 1/5
14/14 [==============================] - 3s 185ms/step - loss: 0.0226 - accuracy: 0.9947 - val_loss: 0.8777 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 180ms/step - loss: 0.0202 - accuracy: 0.9950 - val_loss: 0.4879 - val_accuracy: 0.7382
Epoch 3/5

In [78]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.09564570337533951, 0.11760050803422928], [0.11199367791414261, 0.1584279090166092], [0.10485518723726273, 0.1450893133878708]]
Accuracy for iterations:  [[0.9873571991920471, 0.9762043356895447], [0.9880729913711548, 0.9449793100357056], [0.9883724451065063, 0.9513550996780396]]
F1 for iterations:  [[0.9873610568222886, 0.9761933244015109], [0.9880765019718583, 0.9446595180383826], [0.9883761664384368, 0.9511333102702614]]
Precision for iterations:  [[0.9874329652087068, 0.9762085150909667], [0.9881438005791636, 0.9468737670164072], [0.9884569580419813, 0.9526185930330656]]
Recall for iterations:  [[0.9873572117210553, 0.9762043159194943], [0.98807297905285, 0.9449792923054566], [0.9883724327324783, 0.9513550824674852]]


#### 5C NODE 3 25%

In [79]:
training1 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part1.csv')
training2 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part2.csv')
training3 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part3.csv')
training4 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part4.csv')
training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')

C:\Users\UX430\AppData\Local\Temp/ipykernel_14132/512859027.py:5: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  training5 = pd.read_csv('C:/Users/UX430/Documents/thesis/datasets/UNSW-NB15/UNSW-NB15-Train-Basic-Corr25%35C-Part5.csv')


In [80]:
# Define 
num_nodes = 5
global_updates = 3

# Define model training parameters
loss_fct = "categorical_crossentropy"
metrics = ['accuracy']
local_epochs = 5

In [81]:
def train_local_model(model, node, x_train, y_train): 
    filepath = 'C:/Users/UX430/Documents/thesis/code/models/node'+str(node)+'idcorr255C3krum.hdf5'
    callbacks = [
            keras.callbacks.EarlyStopping(
                monitor = 'val_loss', # Use accuracy to monitor the model
                patience = 10 # Stop after 10 steps with lower accuracy
            ),
            keras.callbacks.ModelCheckpoint(
                filepath = filepath, # file where the checkpoint is saved
                monitor = 'val_loss', # Don't overwrite the saved model unless val_loss is worse
                save_best_only = True)]# Only save model if it is the best
    optimizer = keras.optimizers.Adam(learning_rate=5e-4)
    model.compile(optimizer=optimizer, loss=loss_fct, metrics=metrics)
    history = model.fit(x_train, y_train, epochs=local_epochs, validation_split=0.2, callbacks=callbacks, batch_size=2048)
    return model, history.history['loss'], history.history['accuracy'], history.history['val_loss'], history.history['val_accuracy']

In [82]:
global_model = build_model((24,1,1))

In [83]:
x1, y1 = preprocessing(training1)
x2, y2 = preprocessing(training2)
x3, y3 = preprocessing(training3)
x4, y4 = preprocessing(training4)
x5, y5 = preprocessing(training5)

In [84]:
optimizer = keras.optimizers.Adam(learning_rate=5e-4)

In [85]:
# Values saved each iteration 
loss_it = []
accuracy_it = []
f1_it = []
precision_it = []
recall_it = []

for i in range(global_updates): 
    gradients_list = []
    for node in range(num_nodes): 
        cp = global_model # create a copy of the global model
        if node == 0:
            x, y = x1, y1
        elif node == 1:
            x, y = x2, y2
        elif node == 2:
            x, y = x3, y3
        elif node == 3: 
            x, y = x4, y4
        else: 
            x, y = x5, y5
        local_model, local_loss, local_acc, local_val_loss, local_val_acc = train_local_model(cp, node, x, y)
        with tf.GradientTape() as tape: 
            predictions = local_model(x)
            loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y, predictions))
        gradients = tape.gradient(loss, local_model.trainable_variables)
        gradients_list.append(gradients)

    avg_grad = aggregate(gradients_list,1)
    optimizer.apply_gradients(zip(avg_grad, global_model.trainable_variables)) # apply gradients to global model
    loss_basic, accuracy_basic, f1_basic, precision_basic, recall_basic = evaluation(global_model, xbasic, ybasic) 
    loss_plus, accuracy_plus, f1_plus, precision_plus, recall_plus = evaluation(global_model, xplus, yplus)

    loss_it.append([loss_basic, loss_plus])
    accuracy_it.append([accuracy_basic, accuracy_plus])
    f1_it.append([f1_basic, f1_plus])
    precision_it.append([precision_basic, precision_plus])
    recall_it.append([recall_basic, recall_plus])


global_model.save('C:/Users/UX430/Documents/thesis/code/models/idcorr255C3krum.hdf5')

Epoch 1/5
16/16 [==============================] - 3s 178ms/step - loss: 0.5228 - accuracy: 0.7493 - val_loss: 1.8066 - val_accuracy: 0.0433
Epoch 2/5
16/16 [==============================] - 3s 178ms/step - loss: 0.1801 - accuracy: 0.9798 - val_loss: 3.8603 - val_accuracy: 0.1330
Epoch 3/5
16/16 [==============================] - 3s 169ms/step - loss: 0.0786 - accuracy: 0.9894 - val_loss: 3.5279 - val_accuracy: 0.1339
Epoch 4/5
16/16 [==============================] - 3s 172ms/step - loss: 0.0490 - accuracy: 0.9893 - val_loss: 1.5701 - val_accuracy: 0.1440
Epoch 5/5
16/16 [==============================] - 3s 181ms/step - loss: 0.0360 - accuracy: 0.9896 - val_loss: 1.4584 - val_accuracy: 0.1703
Epoch 1/5
14/14 [==============================] - 2s 170ms/step - loss: 0.0217 - accuracy: 0.9949 - val_loss: 0.8944 - val_accuracy: 0.7270
Epoch 2/5
14/14 [==============================] - 3s 182ms/step - loss: 0.0202 - accuracy: 0.9950 - val_loss: 0.7971 - val_accuracy: 0.7272
Epoch 3/5
14/

In [86]:
print("Loss for iterations: ", loss_it)
print("Accuracy for iterations: ", accuracy_it)
print("F1 for iterations: ", f1_it)
print("Precision for iterations: ", precision_it)
print("Recall for iterations: ", recall_it)

Loss for iterations:  [[0.14725399017333984, 0.20486125349998474], [0.1653129756450653, 0.19722625613212585], [0.23801814019680023, 0.2360980212688446]]
Accuracy for iterations:  [[0.9659791588783264, 0.9168241024017334], [0.9584489464759827, 0.9208202958106995], [0.9373630285263062, 0.8830015063285828]]
F1 for iterations:  [[0.9659263331942735, 0.9157854905944768], [0.9583641109046129, 0.9199154182913], [0.9370426037032887, 0.8802791267222402]]
Precision for iterations:  [[0.9666463352845058, 0.9228811357604424], [0.9594580735964156, 0.9261601643956267], [0.9411268351843823, 0.8968913352484783]]
Recall for iterations:  [[0.965979140494902, 0.9168240935842477], [0.9584489760144906, 0.9208203153382257], [0.9373630547196822, 0.8830015258301243]]
